In [1]:
from sklearn.datasets import make_blobs

import os
import pandas as pd
import numpy as np
import math
import scipy as sp
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt 
from warnings import filterwarnings
filterwarnings('ignore')
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
input_path = '/home/lee43/jupyter-workspace/code/data_sample/'

In [7]:
import os

filelist = os.listdir(input_path) # 현재 디렉토리의 모든 파일명
fileset = [file for file in filelist if file.endswith('.csv')] # csv 파일명만 추출

len(fileset) # csv 파일 개수 출력 (11218)

859

In [8]:
benign_file = [file for file in fileset if file.startswith('0')]
malicious_file = [file for file in fileset if file.startswith('1')]

benign_cnt = len(benign_file)
malicious_cnt = len(malicious_file)

print(benign_cnt, malicious_cnt) # benign 파일과 malicious 파일 개수 출력

358 500


In [9]:
x  = pd.DataFrame()

for file in benign_file:
    df = pd.read_csv(input_path + file)
    data = [] # columns별 연결된 노드 개수 count하여 저장
    
    for col in df.columns:
        data.append(df[col].sum())
    new_df = pd.DataFrame(columns = df.columns)
    new_df.loc[0] = data
    new_df['class'] = 0
    x = pd.concat([x, new_df])

In [10]:
malicious_loop = malicious_cnt / 50; # malicious 50개씩 끊어서 로드

In [11]:
for turn in range(int(malicious_loop)): 
    for file in malicious_file[turn * 50 : (turn + 1) * 50]:
        df = pd.read_csv(input_path + file)
        data = [] # columns별 연결된 노드 개수 count하여 저장
        
        for col in df.columns:
            data.append(df[col].sum())
            
        new_df = pd.DataFrame(columns = df.columns)
        new_df.loc[0] = data
        new_df['class'] = 1
        x = pd.concat([x, new_df])
    print(turn) # (과정 확인용)

0
1
2
3
4
5
6
7
8
9


In [12]:
# 남은 malicious 파일 로드
for file in malicious_file[int(malicious_loop)*50 : ]:
    df = pd.read_csv(input_path + file)
    data = [] # columns별 연결된 노드 개수 count하여 저장
        
    for col in df.columns:
        data.append(df[col].sum())
            
    new_df = pd.DataFrame(columns = df.columns)
    new_df.loc[0] = data
    new_df['class'] = 1
    x = pd.concat([x, new_df])

In [13]:
x.fillna(-1, inplace=True) # 결측치 -1로 채우기
x.tail()

,0,0.1,0.2,0.3,1,1.1,1.2,1.3,10,10.1,...,three,three.1,two,two.1,u,u.1,xl24,xl24.1,zero,zero.1
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [14]:
x.shape

(858, 6544)

In [15]:
x.to_csv(input_path+"preprocess.csv")

In [16]:
x = pd.read_csv(input_path+"preprocess.csv") # 전처리 데이터 불러오기
columns = x.columns
columns = columns.drop("class")
columns = columns.drop("Unnamed: 0") # (index_ignore=True 하지 않아서 Unnamed: 0 컬럼 생성 -> 제거)

# 라벨 분리
y = x["class"]
x = x.drop("class", axis = 1)
x = x.drop("Unnamed: 0", axis = 1) # (index_ignore=True 하지 않아서 Unnamed: 0 컬럼 생성 -> 제거)

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=99)

In [18]:
print(x_train.shape, x_train.ndim)
print(y_train.shape, y_train.ndim)

(686, 6543) 2
(686,) 1


In [19]:
model = Sequential()

model.add(Conv1D(32, 4, strides = 1, activation = 'relu', input_shape=(6543, 1)))

model.add(Conv1D(64, 4, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 6540, 32)          160       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 6537, 64)          8256      
_________________________________________________________________
dropout (Dropout)            (None, 6537, 64)          0         
_________________________________________________________________
dense (Dense)                (None, 6537, 1)           65        
Total params: 8,481
Trainable params: 8,481
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=1)

Epoch 1/10


ValueError: in user code:

    /home/lee43/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/lee43/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/lee43/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/lee43/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/lee43/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home

In [ ]:
#reshape 써서 크기 맞춰보기